In [71]:
import ConnectionConfig as cc
from delta import DeltaTable
from datetime import datetime

In [72]:
cc.setupEnvironment()
spark = cc.startLocalCluster("dimUser")
spark.getActiveSession()

In [73]:
#EXTRACT
run_timestamp = datetime.now()
dt_dimUserCurrent = DeltaTable.forPath(spark,"./spark-warehouse/user_dim_current")

In [75]:
dt_dimUserCurrent.toDF().createOrReplaceTempView("user_dim_current")

In [76]:
spark.sql("select * from user_dim_current").show()

+------+--------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+-------+
|userid|                name|               email|              street|  number|zipcode|                city|country_code|             user_sk|          scd_start|            scd_end|                 md5|current|
+------+--------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+-------+
|     6|      Driessen Anouk|Anouk.Driessen@sc...|   Jan Ockegemstraat|168 0107|   2650|              Edegem|          BE|b1e00544-612a-469...|1990-01-01 00:00:00|2100-12-12 00:00:00|930ae4c084690bf43...|   true|
|     9|            Smit Tim|  Tim.Smit@gmail.com|       Bikschotelaan|     60 |   2600| Berchem (Antwerpen)|          BE|c4ea58f2-8c1c-4f0...|1990-

In [77]:
#EXTRACT
cc.set_connectionProfile("veloDB")

df_operational_users_new = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 1) \
    .option("upperBound", 60000) \
    .load()

df_operational_users_new.createOrReplaceTempView("operational_users")

df_operational_users_new.show(5)


+------+-----------------+--------------------+--------------------+--------+-------+-------------------+------------+
|userid|             name|               email|              street|  number|zipcode|               city|country_code|
+------+-----------------+--------------------+--------------------+--------+-------+-------------------+------------+
|     2|van der Zee Julia|Julia.van.der.Zee...|         Groenplaats|     43 |   2610|Wilrijk (Antwerpen)|          BE|
|     3|  de Boer Ricardo|Ricardo.de.Boer@g...|   Maria Clarastraat|     80 |   2160|          Wommelgem|          BE|
|     4|   Willems Angela|Angela.Willems@te...|Graaf Joseph de P...|     15 |   2900|            Schoten|          BE|
|     5| Heijnen Patricia|Patricia.Heijnen@...|          Meylstraat|    111 |   2540|               Hove|          BE|
|     6|   Driessen Anouk|Anouk.Driessen@sc...|   Jan Ockegemstraat|168 0107|   2650|             Edegem|          BE|
+------+-----------------+--------------------+-

In [78]:
#Transform
df_user_dim_new = spark.sql( """
    SELECT uuid() as source_user_sk,
           userid as source_user_id,
           name as source_user_name,
           email as source_user_email,
           street as source_street,
           number as source_number,
           zipcode as source_zipcode,  
           city as source_city,
           country_code as source_country_code,
           md5(concat(street, number, zipcode, city)) as source_md5
    FROM operational_users
""")

df_user_dim_new.createOrReplaceTempView("user_dim_new")

# Debugging
df_user_dim_new.show()


+--------------------+--------------+--------------------+--------------------+--------------------+-------------+--------------+--------------------+-------------------+--------------------+
|      source_user_sk|source_user_id|    source_user_name|   source_user_email|       source_street|source_number|source_zipcode|         source_city|source_country_code|          source_md5|
+--------------------+--------------+--------------------+--------------------+--------------------+-------------+--------------+--------------------+-------------------+--------------------+
|5949cd9c-4248-406...|             2|   van der Zee Julia|Julia.van.der.Zee...|         Groenplaats|          43 |          2610| Wilrijk (Antwerpen)|                 BE|c03d5fb2d8c9d47ac...|
|f9585d04-b5eb-455...|             3|     de Boer Ricardo|Ricardo.de.Boer@g...|   Maria Clarastraat|          80 |          2160|           Wommelgem|                 BE|18dc3de3ba7fd5861...|
|a56d3c32-a259-46c...|             4|   

In [79]:
detectedChanges = spark.sql("""
    select * from user_dim_new source \
    left outer join user_dim_current dwh on dwh.userid == source.source_user_id and dwh.current == true \
    where dwh.userid is null or dwh.md5 <> source.source_md5
""")

detectedChanges.createOrReplaceTempView("detectedChanges")
detectedChanges.show()


+--------------------+--------------+----------------+-----------------+-------------+-------------+--------------+-----------+-------------------+--------------------+------+----+-----+------+------+-------+----+------------+-------+---------+-------+----+-------+
|      source_user_sk|source_user_id|source_user_name|source_user_email|source_street|source_number|source_zipcode|source_city|source_country_code|          source_md5|userid|name|email|street|number|zipcode|city|country_code|user_sk|scd_start|scd_end| md5|current|
+--------------------+--------------+----------------+-----------------+-------------+-------------+--------------+-----------+-------------------+--------------------+------+----+-----+------+------+-------+----+------------+-------+---------+-------+----+-------+
|3d26eaf8-aad0-457...|         60002|            Nina|   nina@gmail.com|  Breedsteert|           25|          2950|  Antwerpen|                 BE|9f54f467205b634d1...|  NULL|NULL| NULL|  NULL|  NULL|  

In [80]:
df_user_upserts = spark.sql(f"""
    select source_user_sk as user_sk, \
    source_user_id as userid, \
    source_user_name as name, \
    source_user_email as email, \
    source_street as street, \
    source_number as number, \
    source_zipcode as zipcode, \
    source_city as city, \
    source_country_code as country_code, \
    to_timestamp('{run_timestamp}') as scd_start, \
    to_timestamp('2100-12-12', 'yyyy-MM-dd') as scd_end, \
    source_md5 as md5, \
    true as current \
    from detectedChanges \
    union \
    select user_sk, \
    userid, \
    name, \
    email, \
    street,\
    number,\
    zipcode,\
    city,\
    country_code,\
    scd_start,\
    to_timestamp('{run_timestamp}') as scd_end, \
    md5, \
    false \
    from detectedChanges \
    where current is not null
""")

df_user_upserts.createOrReplaceTempView("upserts")
df_user_upserts.show()

+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+
|             user_sk|userid|name|         email|     street|number|zipcode|     city|country_code|           scd_start|            scd_end|                 md5|current|
+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+
|3d26eaf8-aad0-457...| 60002|Nina|nina@gmail.com|Breedsteert|    25|   2950|Antwerpen|          BE|2025-05-08 00:48:...|2100-12-12 00:00:00|9f54f467205b634d1...|   true|
+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+



In [81]:
# DEBUG CODE TO SHOW CONTENT OF UPSERTS
spark.sql("select * from upserts").show()

+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+
|             user_sk|userid|name|         email|     street|number|zipcode|     city|country_code|           scd_start|            scd_end|                 md5|current|
+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+
|3d26eaf8-aad0-457...| 60002|Nina|nina@gmail.com|Breedsteert|    25|   2950|Antwerpen|          BE|2025-05-08 00:48:...|2100-12-12 00:00:00|9f54f467205b634d1...|   true|
+--------------------+------+----+--------------+-----------+------+-------+---------+------------+--------------------+-------------------+--------------------+-------+



In [82]:
#LOAD
spark.sql("""
    MERGE INTO user_dim_current AS target
    USING upserts AS source
    ON target.userid = source.userid AND source.current = false and target.current = true

    WHEN MATCHED THEN UPDATE SET scd_end = source.scd_end, current = source.current
    WHEN NOT MATCHED THEN INSERT (user_sk, userid, name, email, street, number, zipcode, city, country_code, scd_start, scd_end, md5, current) values (source.user_sk, source.userid, source.name, source.email, source.street, source.number, source.zipcode, source.city, source.country_code, source.scd_start, source.scd_end, source.md5, source.current)
""")


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [83]:
dt_dimUserCurrent.toDF().sort("userid", "scd_start").show(100)

+------+--------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------+
|userid|                name|               email|              street|  number|zipcode|                city|country_code|             user_sk|           scd_start|             scd_end|                 md5|current|
+------+--------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------+
|     1|         Bouman Lars|Lars.Bouman@gmail...|        Kanaalstraat|    156 |   2060|           Antwerpen|          BE|7e838551-1cde-482...| 1990-01-01 00:00:00|2025-05-08 00:47:...|7d4821969a29d78e8...|  false|
|     1|         Bouman Lars|Lars.Bouman@gmail...|      De Sossastraat|    156 |   2060|           Antwerpen|          BE|d330235f-c4b4-427.

In [84]:
spark.stop()